In [1]:
from pypdf import PdfReader
reader = PdfReader('data/Official-Results-Mississauga-By-Election-2024.pdf')

In [2]:
page = reader.pages[1].extract_text(extraction_mode="layout", layout_mode_space_vertically=False, layout_mode_scale_weight=1)
print(page)

Summary Results Report                                                                                    OFFICIAL RESULTS
City of Mississauga By-Election 2024
June 10, 2024
1 Forest Avenue PS
Statistics                        TOTAL
Ballots Cast - Total                  819
Ballots Cast - NONRESIDENT            819
Mayor / Maire (ou mairesse)
Vote For 1
                                    TOTAL      VOTE %
Zulfiqar Ali                            1    0.12%
Diya Atassi                             4    0.49%
Brian Crombie                           8    0.98%
Dipika Damerla                         83   10.15%
Stephen Dasko                         508   62.10%
Jamie Dookie                            0    0.00%
Frank Fang                              1    0.12%
Xiaohua Gong                            0    0.00%
Winston Harding                         0    0.00%
Sara Iqbal                              1    0.12%
Syed Jaffery                            0    0.00%
Mohsin Khan                  

In [3]:
page_split = page.split("\n")
page_split

['Summary Results Report                                                                                    OFFICIAL RESULTS',
 'City of Mississauga By-Election 2024',
 'June 10, 2024',
 '1 Forest Avenue PS',
 'Statistics                        TOTAL',
 'Ballots Cast - Total                  819',
 'Ballots Cast - NONRESIDENT            819',
 'Mayor / Maire (ou mairesse)',
 'Vote For 1',
 '                                    TOTAL      VOTE %',
 'Zulfiqar Ali                            1    0.12%',
 'Diya Atassi                             4    0.49%',
 'Brian Crombie                           8    0.98%',
 'Dipika Damerla                         83   10.15%',
 'Stephen Dasko                         508   62.10%',
 'Jamie Dookie                            0    0.00%',
 'Frank Fang                              1    0.12%',
 'Xiaohua Gong                            0    0.00%',
 'Winston Harding                         0    0.00%',
 'Sara Iqbal                              1    0.12%',


In [4]:
location = page_split[page_split.index("June 10, 2024")+1]
location

'1 Forest Avenue PS'

In [5]:
import re
import pandas as pd

table = []
for page in reader.pages[1:]:
    page_text = page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False, layout_mode_scale_weight=1)
    page_split = page_text.split("\n")
    location = page_split[page_split.index("June 10, 2024")+1]
    table_row = {"location": location}
    for row in page_split:
        if row[:6] == "Dipika":
            row_clean = re.sub(' +', ' ', row)
            table_row["Dipika Damerla"] = int(row_clean.split()[2].replace(",", ""))
        if row[:7] == "Stephen":
            row_clean = re.sub(' +', ' ', row)
            table_row["Stephen Dasko"] = int(row_clean.split()[2].replace(",", ""))
        if row[:7] == "Carolyn":
            row_clean = re.sub(' +', ' ', row)
            table_row["Carolyn Parrish"] = int(row_clean.split()[2].replace(",", ""))
        if row[:5] == "Alvin":
            row_clean = re.sub(' +', ' ', row)
            table_row["Alvin Tedjo"] = int(row_clean.split()[2].replace(",", ""))
        if row[:5] == "Total":
            row_clean = re.sub(' +', ' ', row)
            table_row["total"] = int(row_clean.split()[3].replace(",", ""))
        if row[:10] == "Councillor":
            break
    table.append(table_row)

pdf_data = pd.DataFrame(table)
pdf_data

,location,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total
0,1 Forest Avenue PS,83,508,87,114,818
1,1 Kenollie PS,61,289,74,62,510
2,1 Miss. LTC Facility,0,0,0,0,6
3,1 Miss. SC Day 3,78,374,58,62,591
4,1 Miss. SC Dy 4,88,447,80,90,737
...,...,...,...,...,...,...
196,11 Streetsville Heritage Hall Day 3,31,18,75,55,195
197,11 Streetsville Heritage Hall Day 4,38,24,81,61,220
198,11 Streetsville Heritage Hall,119,76,198,156,582
199,11 Streetsville SS,206,129,257,306,979


In [6]:
df = pdf_data.copy()
df["ward"] = df["location"].str.extract(r'(\d+)\s')
df

,location,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total,ward
0,1 Forest Avenue PS,83,508,87,114,818,1
1,1 Kenollie PS,61,289,74,62,510,1
2,1 Miss. LTC Facility,0,0,0,0,6,1
3,1 Miss. SC Day 3,78,374,58,62,591,1
4,1 Miss. SC Dy 4,88,447,80,90,737,1
...,...,...,...,...,...,...,...
196,11 Streetsville Heritage Hall Day 3,31,18,75,55,195,11
197,11 Streetsville Heritage Hall Day 4,38,24,81,61,220,11
198,11 Streetsville Heritage Hall,119,76,198,156,582,11
199,11 Streetsville SS,206,129,257,306,979,11


In [7]:
df["cleaned_location"] = df["location"].str.extract(r'\d+\s(.*)')
df["cleaned_location"] = df["cleaned_location"].map(lambda x: x.replace(' Day 1', ''))
df["cleaned_location"] = df["cleaned_location"].map(lambda x: x.replace(' Day 2', ''))
df["cleaned_location"] = df["cleaned_location"].map(lambda x: x.replace(' Day 3', ''))
df["cleaned_location"] = df["cleaned_location"].map(lambda x: x.replace(' Day 4', ''))
df["cleaned_location"] = df["cleaned_location"].map(lambda x: x.replace(' Dy 3', ''))
df["cleaned_location"] = df["cleaned_location"].map(lambda x: x.replace(' Dy 4', ''))
df

,location,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total,ward,cleaned_location
0,1 Forest Avenue PS,83,508,87,114,818,1,Forest Avenue PS
1,1 Kenollie PS,61,289,74,62,510,1,Kenollie PS
2,1 Miss. LTC Facility,0,0,0,0,6,1,Miss. LTC Facility
3,1 Miss. SC Day 3,78,374,58,62,591,1,Miss. SC
4,1 Miss. SC Dy 4,88,447,80,90,737,1,Miss. SC
...,...,...,...,...,...,...,...,...
196,11 Streetsville Heritage Hall Day 3,31,18,75,55,195,11,Streetsville Heritage Hall
197,11 Streetsville Heritage Hall Day 4,38,24,81,61,220,11,Streetsville Heritage Hall
198,11 Streetsville Heritage Hall,119,76,198,156,582,11,Streetsville Heritage Hall
199,11 Streetsville SS,206,129,257,306,979,11,Streetsville SS


In [8]:
locations = df["cleaned_location"].drop_duplicates().reset_index(drop=True).values.tolist()
print(len(locations))

157


In [9]:
df = df[["cleaned_location", "ward", "Dipika Damerla", "Stephen Dasko", "Carolyn Parrish", "Alvin Tedjo", "total"]]
df = df.rename({"cleaned_location": "location"}, axis=1)
df = df.groupby(["location", "ward"]).sum().reset_index()
df

,location,ward,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total
0,All Saints SS,8,222,245,375,441,1418
1,Amica at City Centre,4,19,7,8,3,49
2,Amica at Erin Mills,8,3,7,7,1,26
3,Artesian Drive PS,8,138,52,410,196,879
4,Beechwood Place RR,3,8,5,8,3,29
...,...,...,...,...,...,...,...
153,Wellbrook Place,2,6,2,8,1,49
154,Westacres PS,1,106,726,118,184,1178
155,Whitehorn PS,6,258,77,483,438,1363
156,Woodlands Library,6,157,58,323,162,761


In [10]:
import json

locations_dict = {loc: "" for loc in locations}
with open("locations.json", "w+") as out:
    out.write(json.dumps(locations_dict))

## Breakdown by ward

In [11]:
df_wards = df.groupby("ward").sum().drop(["location"], axis=1).reset_index()
df_wards["ward"] = df_wards["ward"].astype(int)
df_wards

,ward,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total
0,1,1080,6074,1181,1491,10275
1,10,2261,1022,5492,3892,13740
2,11,2521,1143,3784,2993,11264
3,2,1461,2969,1965,4222,11260
4,3,3463,1900,2696,1795,10906
5,4,3364,1679,4855,3142,14621
6,5,2149,788,6125,2239,12953
7,6,2832,1383,5924,4343,15779
8,7,3589,1819,3705,2305,12306
9,8,2191,2327,3859,4160,13787


In [12]:
df_wards["vote_percentage"] = df_wards["total"] / df_wards["total"].sum() * 100
df_wards

,ward,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total,vote_percentage
0,1,1080,6074,1181,1491,10275,7.338237
1,10,2261,1022,5492,3892,13740,9.812884
2,11,2521,1143,3784,2993,11264,8.044565
3,2,1461,2969,1965,4222,11260,8.041708
4,3,3463,1900,2696,1795,10906,7.788887
5,4,3364,1679,4855,3142,14621,10.442080
6,5,2149,788,6125,2239,12953,9.250821
7,6,2832,1383,5924,4343,15779,11.269104
8,7,3589,1819,3705,2305,12306,8.788744
9,8,2191,2327,3859,4160,13787,9.846451


In [13]:
ward_population = pd.read_csv("data/2016_Census_Data_-_By_Wards.csv")[["Ward", "Pop_"]]
ward_population["Ward"] = ward_population["Ward"].astype(int)
ward_population

,Ward,Pop_
0,4,72710
1,10,74715
2,7,80715
3,3,59055
4,6,78070
5,9,58945
6,1,42605
7,11,60610
8,8,68660
9,2,47375


In [14]:
df_ward_vote_pop = pd.merge(df_wards, ward_population, left_on="ward", right_on="Ward")
df_ward_vote_pop = df_ward_vote_pop.drop("Ward", axis=1)
df_ward_vote_pop

,ward,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total,vote_percentage,Pop_
0,1,1080,6074,1181,1491,10275,7.338237,42605
1,10,2261,1022,5492,3892,13740,9.812884,74715
2,11,2521,1143,3784,2993,11264,8.044565,60610
3,2,1461,2969,1965,4222,11260,8.041708,47375
4,3,3463,1900,2696,1795,10906,7.788887,59055
5,4,3364,1679,4855,3142,14621,10.442080,72710
6,5,2149,788,6125,2239,12953,9.250821,78140
7,6,2832,1383,5924,4343,15779,11.269104,78070
8,7,3589,1819,3705,2305,12306,8.788744,80715
9,8,2191,2327,3859,4160,13787,9.846451,68660


In [15]:
df_ward_vote_pop.to_csv("output/ward_results.csv", index=False)

# Load locations

In [32]:
df_loc = pd.read_json("data/locations-filled.json").transpose()
df_loc.columns = ["coordinates", "is_rr", "address"]
df_loc["coordinates"] = df_loc["coordinates"].str.split(",")
df_loc["latitude"] = df_loc["coordinates"].str[0].astype(float)
df_loc["longitude"] = df_loc["coordinates"].str[1].astype(float)
df_loc.drop("coordinates", axis=1, inplace=True)
df_loc

,is_rr,address,latitude,longitude
Forest Avenue PS,False,"20 Forest Ave, Mississauga, ON L5G 1K7",43.557974,-79.582663
Kenollie PS,False,"1376 Glenwood Dr, Mississauga, ON L5G 2X1",43.557127,-79.598048
Miss. LTC Facility,True,"26 Peter St N, Mississauga, ON L5H 2G7",43.549710,-79.591006
Miss. SC,False,"1389 Cawthra Rd, Mississauga, ON L5G 4L1",43.578656,-79.575499
Port Credit Memorial,False,"40 Stavebank Rd, Mississauga, ON L5G 2T8",43.552719,-79.591434
...,...,...,...,...
St Veronica CS,False,"680 Novo Star Dr, Mississauga, ON L5W 1C7",43.631362,-79.717692
St. Gregory CS,False,"1075 Swinbourne Dr, Mississauga, ON L5V 1B9",43.609409,-79.708183
Streetsville Heritage Hall,False,"327 Queen St S, Mississauga, ON L5M 1M3",43.577515,-79.707977
Streetsville SS,False,"72 Joymar Dr, Mississauga, ON L5M 1G3",43.578539,-79.720460


In [34]:
df_loc_join = pd.merge(df, df_loc, left_on="location", right_index=True)
df_loc_join[["location", "latitude", "longitude", "address", "ward", "Dipika Damerla", "Stephen Dasko", "Carolyn Parrish", "Alvin Tedjo", "total"]].to_csv("output/election_results.csv")
df_loc_join

,location,ward,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total,is_rr,address,latitude,longitude
0,All Saints SS,8,222,245,375,441,1418,False,"4105 Colonial Dr, Mississauga, ON L5L 4E8",43.531740,-79.711113
1,Amica at City Centre,4,19,7,8,3,49,True,,43.587206,-79.647135
2,Amica at Erin Mills,8,3,7,7,1,26,True,"4620 Kimbermount Ave, Mississauga, ON L5M 5W5",43.549483,-79.716787
3,Artesian Drive PS,8,138,52,410,196,879,False,"3325 Artesian Dr, Mississauga, ON L5M 7J8",43.543066,-79.719642
4,Beechwood Place RR,3,8,5,8,3,29,True,,43.628073,-79.604668
...,...,...,...,...,...,...,...,...,...,...,...
153,Wellbrook Place,2,6,2,8,1,49,True,,43.524426,-79.659972
154,Westacres PS,1,106,726,118,184,1178,False,"2165 Breezy Brae Dr, Mississauga, ON L4Y 1N3",43.590239,-79.583471
155,Whitehorn PS,6,258,77,483,438,1363,False,,43.596170,-79.702978
156,Woodlands Library,6,157,58,323,162,761,False,,43.562838,-79.649910


In [18]:
df_loc_join.groupby("is_rr")["total"].sum()

is_rr
False    138979
True       1041
Name: total, dtype: int64

In [19]:
df_loc_join.sort_values("total")

,location,ward,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total,coordinates,is_rr,address
38,Extendicare Miss.,11,0,1,0,0,1,"[43.6323779, -79.7287037]",True,"855 John Watt Blvd, Mississauga, ON L5W 1G2"
47,Heritage Glen,9,2,1,0,2,5,"[43.5829141, -79.7529381]",True,"6515 Glen Erin Dr, Mississauga, ON L5N 8P9"
69,Miss. LTC Facility,1,0,0,0,0,6,"[43.5497559, -79.5933775]",True,
142,Sunrise Senior Living,8,1,0,1,2,6,"[43.6210063, -79.6057242]",True,
14,Cawthra Gardens LTC,4,1,1,2,1,7,"[43.593105, -79.6037121]",True,
...,...,...,...,...,...,...,...,...,...,...
88,River Grove CC,6,741,304,1299,869,3463,"[43.589527, -79.7125299]",False,
20,Clarkson CC,2,456,752,704,1425,3577,"[43.5113661, -79.6504247]",False,
64,Meadowvale CC,9,727,352,1208,1333,4024,"[43.5853306, -79.7567823]",False,"6655 Glen Erin Dr, Mississauga, ON L5N 3L4"
18,Church. Mdws BL,10,688,251,2006,1059,4355,"[43.5523473, -79.7476441]",False,"3801 Thomas St, Mississauga, ON L5M 7G2"


In [ ]:
# Retirement home turnout
df_rr = df_loc_join[df_loc_join["is_rr"] == True]
df_rr[["Dipika Damerla", "Stephen Dasko", "Carolyn Parrish", "Alvin Tedjo", "total"]].sum()

## Plot results on a map

In [36]:
df_loc_join["winner"] = df_loc_join[["Dipika Damerla", "Stephen Dasko", "Carolyn Parrish", "Alvin Tedjo"]].idxmax(axis=1)
df_loc_join["vote_percentage"] = df_loc_join[["Dipika Damerla", "Stephen Dasko", "Carolyn Parrish", "Alvin Tedjo"]].values.max(1) / df_loc_join["total"]
df_loc_join

,location,ward,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total,is_rr,address,latitude,longitude,winner,vote_percentage
0,All Saints SS,8,222,245,375,441,1418,False,"4105 Colonial Dr, Mississauga, ON L5L 4E8",43.531740,-79.711113,Alvin Tedjo,0.311001
1,Amica at City Centre,4,19,7,8,3,49,True,,43.587206,-79.647135,Dipika Damerla,0.387755
2,Amica at Erin Mills,8,3,7,7,1,26,True,"4620 Kimbermount Ave, Mississauga, ON L5M 5W5",43.549483,-79.716787,Stephen Dasko,0.269231
3,Artesian Drive PS,8,138,52,410,196,879,False,"3325 Artesian Dr, Mississauga, ON L5M 7J8",43.543066,-79.719642,Carolyn Parrish,0.466439
4,Beechwood Place RR,3,8,5,8,3,29,True,,43.628073,-79.604668,Dipika Damerla,0.275862
...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,Wellbrook Place,2,6,2,8,1,49,True,,43.524426,-79.659972,Carolyn Parrish,0.163265
154,Westacres PS,1,106,726,118,184,1178,False,"2165 Breezy Brae Dr, Mississauga, ON L4Y 1N3",43.590239,-79.583471,Stephen Dasko,0.616299
155,Whitehorn PS,6,258,77,483,438,1363,False,,43.596170,-79.702978,Carolyn Parrish,0.354365
156,Woodlands Library,6,157,58,323,162,761,False,,43.562838,-79.649910,Carolyn Parrish,0.424442


In [43]:
import plotly.express as px

# Colours to use for plot
winner_colours = {
    "Dipika Damerla": "gold",
    "Stephen Dasko": "blue",
    "Carolyn Parrish": "red",
    "Alvin Tedjo": "green"
}

px.set_mapbox_access_token(open(".mapbox_token").read())
# fig = px.scatter_mapbox(df_loc_join, lat='latitude', lon='longitude', color='winner', color_discrete_map=winner_colours,
#                      hover_name="location", size='total', size_max=20, zoom=9, width=600, height=500,
#                      hover_data=["Dipika Damerla", "Stephen Dasko", "Carolyn Parrish", "Alvin Tedjo", "total"],
#                      title='<b>2024 Mississauga By-Election Polling Results</b> <br>Larger bubbles mean more votes at that location')
fig = px.scatter_geo(df_loc_join, lat='latitude', lon='longitude', color='winner', color_discrete_map=winner_colours,
                     hover_name="location", size='total', size_max=20, width=600, height=500,
                     hover_data=["Dipika Damerla", "Stephen Dasko", "Carolyn Parrish", "Alvin Tedjo", "total"],
                     title='<b>2024 Mississauga By-Election Polling Results</b> <br>Larger bubbles mean more votes at that location')
fig.update_layout(
        geo = dict(
            projection_scale=500, #this is kind of like zoom
            center=dict(lat=43.6014499, lon=-79.6750527), # this will center on the point
            resolution=50
        ))
fig.show()

In [25]:
fig.write_html("html/full_opacity.html")

In [47]:
# Change size to be Alvin's votes
fig2 = px.scatter_geo(df_loc_join, lat='latitude', lon='longitude', color='winner', color_discrete_map=winner_colours,
                     hover_name="location", size='Alvin Tedjo', size_max=20, width=600, height=500, #zoom=9,
                     hover_data=["Dipika Damerla", "Stephen Dasko", "Carolyn Parrish", "Alvin Tedjo", "total"],
                     title='<b>2024 Mississauga By-Election Polling Results</b> <br>Larger bubbles mean more votes for Alvin at that location')
fig2.update_layout(
        geo = dict(
            projection_scale=500, #this is kind of like zoom
            center=dict(lat=43.6014499, lon=-79.6750527), # this will center on the point
            resolution=50
        ))

This is too pale, remove RR to fix scale

Also, reformat the data to work with graph_objects so that we can make the opacity correspond to the winning vote margin

In [54]:
winner_colours_rgb = {
    "Dipika Damerla": "rgba(255, 215, 0, ",
    "Stephen Dasko": "rgba(0, 0, 255, ",
    "Carolyn Parrish": "rgba(255, 0, 0, ",
    "Alvin Tedjo": "rgba(0, 128, 0, "
}
df_go = df_loc_join.copy()
df_go["marker_color"] = df_go.apply(lambda row: winner_colours_rgb[row["winner"]] + str(row["vote_percentage"]) + ")", axis=1)
df_go["marker_size"] = 20 * (df_go["total"] / df_go["total"].max()) ** 0.5
df_go["alvin_marker_size"] = 20 * (df_go["Alvin Tedjo"] / df_go["Alvin Tedjo"].max()) ** 0.5
df_go_non_rr = df_go[df_go["is_rr"] == False].copy()
df_go_non_rr["vote_percentage"] = df_go_non_rr[["Dipika Damerla", "Stephen Dasko", "Carolyn Parrish", "Alvin Tedjo"]].values.max(1) / df_go_non_rr["total"]
df_go_non_rr["hover_text"] = df_go_non_rr.apply(lambda row: f"""Winner: {row["winner"]}<br>
Total votes: {row["total"]}<br>
Dipika Damerla: {row["Dipika Damerla"]}<br>
Stephen Dasko: {row["Stephen Dasko"]}<br>
Carolyn Parrish: {row["Carolyn Parrish"]}<br>
Alvin Tedjo: {row["Alvin Tedjo"]}<br>
Winner vote %: {100*row["vote_percentage"]:.2f}
""", axis=1)
df_go_non_rr["vote_percentage"] = df_go_non_rr["vote_percentage"] - df_go_non_rr["vote_percentage"].min() + 0.001
df_go_non_rr["vote_percentage"] = (df_go_non_rr["vote_percentage"] / df_go_non_rr["vote_percentage"].max()) ** 0.25
df_go_non_rr["marker_color"] = df_go_non_rr.apply(lambda row: winner_colours_rgb[row["winner"]] + str(row["vote_percentage"]) + ")", axis=1)
df_go_non_rr.sort_values("vote_percentage")

,location,ward,Dipika Damerla,Stephen Dasko,Carolyn Parrish,Alvin Tedjo,total,is_rr,address,latitude,longitude,winner,vote_percentage,marker_color,marker_size,alvin_marker_size,hover_text
138,Sts. Martha and Mary CS,3,282,250,292,201,1082,False,,43.636180,-79.602457,Carolyn Parrish,0.230843,"rgba(255, 0, 0, 0.23084269670863025)",9.967809,7.511395,Winner: Carolyn Parrish<br>\nTotal votes: 1082...
134,St. Timothy SS,7,148,120,159,88,557,False,"2214 Cliff Rd, Mississauga, ON L5A 2N9",43.579998,-79.600518,Carolyn Parrish,0.465864,"rgba(255, 0, 0, 0.46586427484481296)",7.151772,4.970086,Winner: Carolyn Parrish<br>\nTotal votes: 557<...
24,Cooksville Lifecare CTR,7,1,1,4,1,14,False,"55 Queensway W, Mississauga, ON L5B 1B5",43.573321,-79.609884,Carolyn Parrish,0.467655,"rgba(255, 0, 0, 0.46765475779109994)",1.133836,0.529813,Winner: Carolyn Parrish<br>\nTotal votes: 14<b...
9,Briarwood PS,4,213,109,221,158,773,False,,43.602361,-79.620001,Carolyn Parrish,0.468932,"rgba(255, 0, 0, 0.46893228973377055)",8.425114,6.659645,Winner: Carolyn Parrish<br>\nTotal votes: 773<...
74,Munden Park PS,7,139,152,163,81,569,False,"515 Tedwyn Dr, Mississauga, ON L5A 1J8",43.583088,-79.593115,Carolyn Parrish,0.472797,"rgba(255, 0, 0, 0.47279735436408815)",7.228400,4.768316,Winner: Carolyn Parrish<br>\nTotal votes: 569<...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,Miss. SC,1,315,1644,283,356,2728,False,"1389 Cawthra Rd, Mississauga, ON L5G 4L1",43.578656,-79.575499,Stephen Dasko,0.986683,"rgba(0, 0, 255, 0.9866827519058263)",15.827351,9.996491,Winner: Stephen Dasko<br>\nTotal votes: 2728<b...
81,Port Credit SS,1,91,544,101,120,886,False,"70 Mineola Rd E, Mississauga, ON L5G 2E5",43.562586,-79.590925,Stephen Dasko,0.994971,"rgba(0, 0, 255, 0.9949705007140692)",9.019925,5.803810,Winner: Stephen Dasko<br>\nTotal votes: 886<br...
154,Westacres PS,1,106,726,118,184,1178,False,"2165 Breezy Brae Dr, Mississauga, ON L4Y 1N3",43.590239,-79.583471,Stephen Dasko,0.996626,"rgba(0, 0, 255, 0.9966264435820429)",10.400607,7.186732,Winner: Stephen Dasko<br>\nTotal votes: 1178<b...
87,Ridgewood PS,5,51,10,269,49,435,False,,43.715266,-79.640917,Carolyn Parrish,0.998123,"rgba(255, 0, 0, 0.9981233314591352)",6.320198,3.708691,Winner: Carolyn Parrish<br>\nTotal votes: 435<...


In [94]:
import plotly.graph_objects as go

mapbox_access_token = open(".mapbox_token").read()
fig3 = go.Figure(data=go.Scattermapbox(
        lon = df_go_non_rr['longitude'],
        lat = df_go_non_rr['latitude'],
        text = df_go_non_rr["hover_text"],
        mode = 'markers',
        marker_color = df_go_non_rr['marker_color'],
        marker_size = df_go_non_rr['marker_size']
        ))
fig3.update_layout(
        title_text = "<b>2024 Mississauga By-Election Polling Results</b> <br>" +
        "Bubble size shows how many votes were cast at that station.<br>" +
        "Bubble transparency shows how much of the vote the winning<br>" +
        "candidate got (darker = higher % for winning candidate)<br><br>" +
        "<b>Legend</b><br>" +
        '<span style="color: gold;">Dipika Damerla</span><br>' +
        '<span style="color: blue;">Stephen Dasko</span><br>' +
        '<span style="color: red;">Carolyn Parrish</span><br>' +
        '<span style="color: green;">Alvin Tedjo</span><br>',
        width = 600,
        height = 500,
        mapbox=dict(
                accesstoken=mapbox_access_token,
                bearing=0,
                center=go.layout.mapbox.Center(
                lat=43.6014499,
                lon=-79.6750527
                ),
                pitch=0,
                zoom=9
    ),
    margin={"r": 20, "t": 150, "l": 170, "b": 20}
    )
fig3.show()

In [95]:
fig3.write_html("html/index.html")

In [97]:
winner_colours_rgb = {
    "Dipika Damerla": "rgba(255, 215, 0, ",
    "Stephen Dasko": "rgba(0, 0, 255, ",
    "Carolyn Parrish": "rgba(255, 0, 0, ",
    "Alvin Tedjo": "rgba(0, 128, 0, "
}

fig4 = go.Figure(data=go.Scattermapbox(
        lon = df_go_non_rr['longitude'],
        lat = df_go_non_rr['latitude'],
        text = df_go_non_rr["hover_text"],
        mode = 'markers',
        marker_color = df_go_non_rr['marker_color'],
        marker_size = df_go_non_rr['alvin_marker_size'],
        ))
fig4.update_layout(
        title_text = "<b>2024 Mississauga By-Election Polling Results</b> <br>" +
        "Bubble size shows how many votes Alvin got at that station.<br>" +
        "Bubble transparency shows how much of the vote the winning<br>" +
        "candidate got (darker = higher % for winning candidate)<br><br>" +
        "<b>Legend</b><br>" +
        '<span style="color: gold;">Dipika Damerla</span><br>' +
        '<span style="color: blue;">Stephen Dasko</span><br>' +
        '<span style="color: red;">Carolyn Parrish</span><br>' +
        '<span style="color: green;">Alvin Tedjo</span><br>',
        width = 600,
        height = 500,
        mapbox=dict(
                accesstoken=mapbox_access_token,
                bearing=0,
                center=go.layout.mapbox.Center(
                lat=43.6014499,
                lon=-79.6750527
                ),
                pitch=0,
                zoom=9
        ),
        margin={"r": 20, "t": 150, "l": 170, "b": 20})
fig4.show()

In [98]:
fig4.write_html("html/alvin_votes.html")